In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare
import xgboost
from google.colab import drive
import pickle
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, average_precision_score, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
import time
np.seterr(divide='ignore', invalid='ignore')
!pip install scikit-posthocs
import scikit_posthocs as sp


     |████████████████████████████████| 43 kB 1.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-posthocs: filename=scikit_posthocs-0.6.7-py3-none-any.whl size=37903 sha256=fd63bf498ab11889954cdaa2240be20913d02f7177398aafd5c7d221a35891ea
  Stored in directory: /root/.cache/pip/wheels/b8/21/e6/f39794d4a6ee3d3cc5146ca80b5cd949452ad4a8fde9f6b9fc
Successfully built scikit-posthocs


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [ ]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [91]:
xgboost_five=pd.read_csv("/content/gdrive/MyDrive/Studying/Computational_Learning/HW4/results_summary/XGBoost0.1.csv").drop(columns=["Unnamed: 0"])
pi_model_five=pd.read_csv("/content/gdrive/MyDrive/Studying/Computational_Learning/HW4/results_summary/False0.1All_Summary.csv").rename(columns={"inference_time [seconds]":"inference time 1000 samples [seconds]", "train_time [seconds]":"train time [seconds]"})
improved_model=pd.read_csv("/content/gdrive/MyDrive/Studying/Computational_Learning/HW4/results_summary/True0.1All_Summary.csv").rename(columns={"inference_time [seconds]":"inference time 1000 samples [seconds]", "train_time [seconds]":"train time [seconds]"})

In [92]:
friedmanchisquare(xgboost_five["ROC_AUC"], pi_model_five["ROC_AUC"], improved_model["ROC_AUC"])

FriedmanchisquareResult(statistic=27.69999999999999, pvalue=9.660985396665186e-07)

In [93]:
data = np.array([xgboost_five["ROC_AUC"], pi_model_five["ROC_AUC"], improved_model["ROC_AUC"]])
nim=pd.DataFrame(sp.posthoc_nemenyi_friedman(data.T))
nim


,0,1,2
0,1.000000,0.691266,0.001
1,0.691266,1.000000,0.001
2,0.001000,0.001000,1.000


In [94]:
print(improved_model["ROC_AUC"].mean())
print(pi_model_five["ROC_AUC"].mean())
print(xgboost_five["ROC_AUC"].mean())

0.936071451620663
0.83906532614507
0.8107794390118472


In [95]:
a=improved_model.mean()
b=improved_model.std()/(10**0.5)
c1=pd.DataFrame(["{:.3f} + {:.3f}".format(a.iloc[i], b.iloc[i]) for i in range(10)], index=a.index)
c1=c1.T
c1.loc[0, "model"]="improved pi model"
a=pi_model_five.mean()
b=pi_model_five.std()/(10**0.5)
c2=pd.DataFrame(["{:.3f} + {:.3f}".format(a.iloc[i], b.iloc[i]) for i in range(10)], index=a.index)
c2=c2.T
c2.loc[0, "model"]="pi model"
a=xgboost_five.mean()
b=xgboost_five.std()/(10**0.5)
c3=pd.DataFrame(["{:.3f} + {:.3f}".format(a.iloc[i], b.iloc[i]) for i in range(8)], index=a.index)
c3=c3.T
c3.loc[0, "model"]="xgboost model"

In [96]:
c=pd.concat([c1, c2, c3])
c

,Accuracy,ROC_AUC,PR_AUC,Precision,TPR,FPR,inference time 1000 samples [seconds],train time [seconds],noise,drop_rate,model
0,0.827 + 0.045,0.936 + 0.031,0.846 + 0.049,0.846 + 0.036,0.749 + 0.061,0.105 + 0.050,0.266 + 0.097,55.295 + 9.613,0.155 + 0.003,0.142 + 0.005,improved pi model
0,0.681 + 0.078,0.839 + 0.041,0.665 + 0.078,0.751 + 0.035,0.590 + 0.082,0.137 + 0.052,0.222 + 0.108,65.344 + 11.333,0.155 + 0.004,0.144 + 0.003,pi model
0,0.712 + 0.080,0.811 + 0.058,0.682 + 0.094,0.762 + 0.044,0.633 + 0.086,0.118 + 0.048,0.157 + 0.095,0.503 + 0.205,NaN,NaN,xgboost model
